**Introduction to Types and Generic Programming**

[Quant Econ](https://julia.quantecon.org/getting_started_julia/introduction_to_types.html)

May 22 2020

Taisei Noda

**NOTE**

* Exercise 6 has not been completed (May/27/2020).

In [1]:
using InstantiateFromURL
# optionally add arguments to force installation: instantiate = true, precompile = true
github_project("QuantEcon/quantecon-notebooks-julia", version = "0.7.0")

Activated /Users/taisei/Project.toml
Info quantecon-notebooks-julia 0.5.0 activated, 0.7.0 requested


In [2]:
using LinearAlgebra, Statistics

# Finding and Interpreting Types

## Finding the Type

In [3]:
@show typeof(1)
@show typeof(1.0);

typeof(1) = Int64
typeof(1.0) = Float64


In [4]:
x=1
typeof(x)

Int64

## Parametric Types vs Concrete Types

Array, Tuple, Complex..etc.

In [5]:
@show typeof(1.0+1im)
@show typeof(ones(2,2));

typeof(1.0 + 1im) = Complex{Float64}
typeof(ones(2, 2)) = Array{Float64,2}


* parametruc type

Array{Type,Dimension}

In [6]:
x =(1,2.0,"test")
@show typeof(x);

typeof(x) = Tuple{Int64,Float64,String}


In [7]:
x=(a=1,b=2.0,c="test")
@show typeof(x);

typeof(x) = NamedTuple{(:a, :b, :c),Tuple{Int64,Float64,String}}


* simple(?)

In [8]:
typeof(:a)

Symbol

## Variables, Types, and Values

In [9]:
x=42
@show typeof(x);

typeof(x) = Int64


In [10]:
x=42.0
typeof(x)

Float64

# The Type Hierarchy

## Abstract vs Concrete Types

* Concrete type

Concrete types are types that we can instantiate – i.e., pair with data in memory.

* Abstract type

We will now examine abstract types that cannot be instantiated (e.g., Real, AbstractFloat).

While you will never have a Real number directly in memory, the abstract types help us organize and work with related concrete types.

## Subtypes and Supertypes

* <:

In [11]:
@show Float64 <: Real
@show Int64 <: Real
@show Complex{Float64} <: Real
@show Array <: Real;

Float64 <: Real = true
Int64 <: Real = true
Complex{Float64} <: Real = false
Array <: Real = false


In [12]:
@show Real <: Number
@show Float64 <: Number
@show Int64 <: Number
@show Complex{Float64} <: Number;

Real <: Number = true
Float64 <: Number = true
Int64 <: Number = true
Complex{Float64} <: Number = true


In [13]:
Number <: Any

true

Float64<:Real<:Number

In [14]:
using Base: show_supertypes  # import the function from the `Base` package

show_supertypes(Int64)

Int64 <: Signed <: Integer <: Real <: Number <: Any

In [15]:
@show subtypes(Real)
@show subtypes(AbstractFloat);

subtypes(Real) = Any[AbstractFloat, AbstractIrrational, Integer, Rational]
subtypes(AbstractFloat) = Any[BigFloat, Float16, Float32, Float64]


# Deducing and Declaring Types

In [16]:
x1=[1,2,3]
x2=[1.0,2.0,3.0]
@show typeof(x1)
@show typeof(x2)

typeof(x1) = Array{Int64,1}
typeof(x2) = Array{Float64,1}


Array{Float64,1}

In [17]:
f(y)=2y #define some function
x=[1,2,3]
z=f(x)

3-element Array{Int64,1}:
 2
 4
 6

## Good Practices for Functions and Variable Types

* poor style: lack of **type stability** (consistentcy in types)

In [18]:
x=[1.0,"test",1]

3-element Array{Any,1}:
 1.0
  "test"
 1

In [19]:
function f(x)
    if x > 0
        return 1.0
    else
        return 0  # probably meant `0.0`
    end
end

@show f(1)
@show f(-1)

f(1) = 1.0
f(-1) = 0


0

* the type of the output is inconsistent.

## Manually Declaring Function and Variable Types

In [20]:
function f(x, A)
    b = [5.0, 6.0]
    return A * x .+ b
end

val = f([0.1, 2.0], [1.0 2.0; 3.0 4.0])

2-element Array{Float64,1}:
  9.1
 14.3

In [21]:
function f2(x::Vector{Float64}, A::Matrix{Float64})::Vector{Float64}
    # argument and return types
    b::Vector{Float64} = [5.0, 6.0]
    return A * x .+ b
end

val = f2([0.1; 2.0], [1.0 2.0; 3.0 4.0])

2-element Array{Float64,1}:
  9.1
 14.3

* **"::TYPE{TYPE}""**

In [22]:
@show f([0.1; 2.0], [1 2; 3 4])
@show f([0.1; 2.0], Diagonal([1.0, 2.0]))

# f2([0.1; 2.0], [1 2; 3 4]) # not a `Float64` "no method mathing"
# f2([0.1; 2.0], Diagonal([1.0, 2.0])) # not a `Matrix{Float64}`

f([0.1; 2.0], [1 2; 3 4]) = [9.1, 14.3]
f([0.1; 2.0], Diagonal([1.0, 2.0])) = [5.1, 10.0]


2-element Array{Float64,1}:
  5.1
 10.0

# Creating New Types

## Syntax for Creating Concrete Types

* "struct"

Bad Example

In [23]:
struct FooNotTyped   # immutable by default, use `mutable struct` otherwise
    a
    b
    c
end

In [24]:
FooNotTyped

FooNotTyped

Good Example

**NOTE**:cannot redefinition.

In [25]:
struct Zoo
    a::Float64
    b::Int64
    c::Vector{Float64}
end

In [26]:
foo_nt = FooNotTyped(2.0, 3, [1.0, 2.0, 3.0])  # new `FooNotTyped`
foo = Foo(2.0, 3, [1.0, 2.0, 3.0]) # creates a new `Foo`

@show typeof(foo)
@show foo.a       # get the value for a field
@show foo.b
@show foo.c;

# foo.a = 2.0     # fails since it is immutable

UndefVarError: UndefVarError: Foo not defined

In [27]:
@show typeof(foo_nt)
@show foo_nt.a
@show foo_nt.b
@show foo_nt.c;

typeof(foo_nt) = FooNotTyped
foo_nt.a = 2.0
foo_nt.b = 3
foo_nt.c = [1.0, 2.0, 3.0]


**NOTE**:Difference? What is the problem?

=>see the next section. One of the promlems is low performance.

In [28]:
foo.a=2.0 #immutable

UndefVarError: UndefVarError: foo not defined

## Issues with Type Declarations

Was it necessary to manually declare the types a::Float64 in the above struct?

The answer, in practice, is usually yes.

In [29]:
struct Foo2
    a::Float64
    b::Integer  # BAD! Not a concrete type
    c::Vector{Real}  # BAD! Not a concrete type
end

In [30]:
f(x) = x.a + x.b + sum(x.c) # use the type
a = 2.0
b = 3
c = [1.0, 2.0, 3.0]
foo = Foo(a, b, c)
@show f(foo)   # call with the foo, no problem

# some other typed for the values
a = 2   # not a floating point but `f()` would work
b = 3
c = [1.0, 2.0, 3.0]'   # transpose is not a `Vector` but `f()` would work
# foo = Foo(a, b, c)   # fails to compile

# works with `NotTyped` version, but low performance
foo_nt = FooNotTyped(a, b, c)
@show f(foo_nt);

UndefVarError: UndefVarError: Foo not defined

## Declaring Parametric Types (Advanced)

This approach is flexible and highly performed.

In [31]:
struct Foo3{T1, T2, T3}
    a::T1   # could be any type
    b::T2
    c::T3
end

# works fine
a = 2
b = 3
c = [1.0, 2.0, 3.0]'    # transpose is not a `Vector` but `f()` would work
foo = Foo3(a, b, c)
@show typeof(foo)
f(foo)

typeof(foo) = Foo3{Int64,Int64,Adjoint{Float64,Array{Float64,1}}}


11.0

However, this is too flexible. The below is fairly constrained, using "**<:**"

In [32]:
struct Foo4{T1 <: Real, T2 <: Real, T3 <: AbstractVecOrMat{<:Real}}
    a::T1
    b::T2
    c::T3  # should check dimensions as well
end
foo = Foo4(a, b, c)  # no problem, and high performance
@show typeof(foo)
f(foo)

typeof(foo) = Foo4{Int64,Int64,Adjoint{Float64,Array{Float64,1}}}


11.0

## Keyword Argument Constructors (Advanced)

* using **Parameters.jl**

In [33]:
using Parameters

@with_kw  struct Foo5
    a::Float64 = 2.0     # adds default value
    b::Int64
    c::Vector{Float64}
end

foo = Foo5(a = 0.1, b = 2, c = [1.0, 2.0, 3.0])
foo2 = Foo5(c = [1.0, 2.0, 3.0], b = 2)  # rearrange order, uses default values

@show foo
@show foo2

function f(x)
    @unpack a, b, c = x     # can use `@unpack` on any struct
    return a + b + sum(c)
end

f(foo)

┌ Info: Precompiling Parameters [d96e819e-fc66-5662-9728-84c9c7592b0a]
└ @ Base loading.jl:1260


foo = Foo5
  a: Float64 0.1
  b: Int64 2
  c: Array{Float64}((3,)) [1.0, 2.0, 3.0]

foo2 = Foo5
  a: Float64 2.0
  b: Int64 2
  c: Array{Float64}((3,)) [1.0, 2.0, 3.0]



8.1

## Tips and Tricks for Writing Generic Functions

* There is major advantage to never declaring a type unless it is absolutely necessary.
* The main place where it is necessary is designing code around **multiple dispatch**.

In [34]:
# BAD
x = [5.0, 6.0, 2.1]

function g(x::Array{Float64, 1})   # not generic!
    y = zeros(length(x))   # not generic, hidden float!
    z = Diagonal(ones(length(x)))  # not generic, hidden float!
    q = ones(length(x))
    y .= z * x + q
    return y
end

g(x)

3-element Array{Float64,1}:
 6.0
 7.0
 3.1

In [35]:
# GOOD
function g2(x)  # or `x::AbstractVector`
    y = similar(x)
    z = I
    q = ones(eltype(x), length(x))  # or `fill(one(x), length(x))`
    y .= z * x + q
    return y
end

g2(x)

3-element Array{Float64,1}:
 6.0
 7.0
 3.1

Notice that the good example does not specify the type of the argument for function g.

In [36]:
x::AbstractVector

3-element Array{Float64,1}:
 5.0
 6.0
 2.1

In [37]:
eltype(x)

Float64

In [38]:
typeof(x)

Array{Float64,1}

* Key: "similar", "eltype", and "typeof"

In [39]:
? eltype

search: eltype fieldtype fieldtypes



```
eltype(type)
```

Determine the type of the elements generated by iterating a collection of the given `type`. For dictionary types, this will be a `Pair{KeyType,ValType}`. The definition `eltype(x) = eltype(typeof(x))` is provided for convenience so that instances can be passed instead of types. However the form that accepts a type argument should be defined for new types.

# Examples

```jldoctest
julia> eltype(fill(1f0, (2,2)))
Float32

julia> eltype(fill(0x1, (2,2)))
UInt8
```


In [40]:
function g(x)
    y = similar(x)
    for i in eachindex(x)
        y[i] = x[i]^2      # could broadcast
    end
    return y
end

g([BigInt(1), BigInt(2)])

2-element Array{BigInt,1}:
 1
 4

In [41]:
@show typeof([1.0, 2.0, 3.0])
@show eltype([1.0, 2.0, 3.0]);

typeof([1.0, 2.0, 3.0]) = Array{Float64,1}
eltype([1.0, 2.0, 3.0]) = Float64


* **Beware of hidden floating points!!**

In [42]:
@show typeof(ones(3))
@show typeof(ones(Int64, 3))
@show typeof(zeros(3))
@show typeof(zeros(Int64, 3));

typeof(ones(3)) = Array{Float64,1}
typeof(ones(Int64, 3)) = Array{Int64,1}
typeof(zeros(3)) = Array{Float64,1}
typeof(zeros(Int64, 3)) = Array{Int64,1}


In [43]:
@show typeof(1)
@show typeof(1.0)
@show typeof(BigFloat(1.0))
@show typeof(one(BigFloat))  # gets multiplicative identity, passing in type
@show typeof(zero(BigFloat))

x = BigFloat(2)

@show typeof(one(x))  # can call with a variable for convenience
@show typeof(zero(x));

typeof(1) = Int64
typeof(1.0) = Float64
typeof(BigFloat(1.0)) = BigFloat
typeof(one(BigFloat)) = BigFloat
typeof(zero(BigFloat)) = BigFloat
typeof(one(x)) = BigFloat
typeof(zero(x)) = BigFloat


In [44]:
# ACCEPTABLE
function g(x::AbstractFloat)
    return x + 1.0   # assumes `1.0` can be converted to something compatible with `typeof(x)`
end

x = BigFloat(1.0)

@show typeof(g(x));  # this has "promoted" the `1.0` to a `BigFloat`

typeof(g(x)) = BigFloat


"**BigFloat**": The BigInt and BigFloat types are available in Julia for arbitrary precision integer and floating point numbers respectively. NOTE:Int64 and Float64 have upper and lower bounds.

In [45]:
x=typemax(Int64)

9223372036854775807

In [46]:
x=x+1

-9223372036854775808

In [47]:
typeof(x)

Int64

In [48]:
x

-9223372036854775808

In [49]:
x=typemin(Int64)

-9223372036854775808

In [50]:
x=x-1

9223372036854775807

In [51]:
x=typemax(BigFloat)

Inf

In [52]:
typeof(x)

BigFloat

In [53]:
x=1

1

In [54]:
typeof(x)

Int64

In [55]:
typeof(one(x))

Int64

* But sometimes assuming promotion is not enough.

In [56]:
# BAD
function g2(x::AbstractFloat)
    if x > 0.0   # can't efficiently call with `x::Integer`
        return x + 1.0   # OK - assumes you can promote `Float64` to `AbstractFloat`
    otherwise
        return 0   # BAD! Returns a `Int64`
    end
end

x = BigFloat(1.0)
x2 = BigFloat(-1.0)

@show typeof(g2(x))
@show typeof(g2(x2))  # type unstable

typeof(g2(x)) = BigFloat
typeof(g2(x2)) = Nothing


Nothing

In [57]:
# GOOD
function g3(x) #don't declear
    if x > zero(x)   # any type with an additive identity
        return x + one(x)  # more general but less important of a change
    otherwise
        return zero(x)
    end
end

@show typeof(g3(x))
@show typeof(g3(x2));  # type stable

typeof(g3(x)) = BigFloat
typeof(g3(x2)) = Nothing


In [58]:
y=g3(-1)
@show typeof(y)

typeof(y) = Nothing


Nothing

In [59]:
typeof(zero(-1))

Int64

If any of the functions you write or call are not precise enough, then it may break the chain.

This is all the more reason to exploit carefully designed packages rather than “do-it-yourself”.

## A Digression on Style and Naming

* Lesson: “be aware of types, but avoid declaring them”.

* Provide easy to read code with minimal “syntactic noise” and a clear correspondence to the math.
* Ensure that code is sufficiently generic to exploit other packages and types.
* Avoid common mistakes and unnecessary performance degradations.


# Introduction to Multiple Dispatch

**Multiple dispatch**: The same function name can do different things depending on the underlying types.

In [60]:
@show abs(-1)   # `Int64`
@show abs(-1.0)  # `Float64`
@show abs(0.0 - 1.0im);  # `Complex{Float64}`

abs(-1) = 1
abs(-1.0) = 1.0
abs(0.0 - 1.0im) = 1.0


In [61]:
function ourabs(x::Real)
    if x > zero(x)   # note, not 0!
        return x
    else
        return -x
    end
end

function ourabs(x::Complex)
    sqrt(real(x)^2 + imag(x)^2)
end

@show ourabs(-1)   # `Int64`
@show ourabs(-1.0) # `Float64`
@show ourabs(1.0 - 2.0im);  # `Complex{Float64}`

ourabs(-1) = 1
ourabs(-1.0) = 1.0
ourabs(1.0 - 2.0im) = 2.23606797749979


In [62]:
x = -2//3  # `Rational` number, -2/3
@show typeof(x)
@show ourabs(x);

typeof(x) = Rational{Int64}
ourabs(x) = 2//3


In [63]:
? 2//3

```
//(num, den)
```

Divide two integers or rational numbers, giving a [`Rational`](@ref) result.

# Examples

```jldoctest
julia> 3 // 5
3//5

julia> (3 // 5) // (2 // 1)
3//10
```


## Multiple Dispatch in Algorithms (Advanced)

In [64]:
x = range(0.0, 1.0, length = 20)
x_2 = 1:1:20   # if integers

@show typeof(x)
@show typeof(x_2)
@show supertype(typeof(x))

typeof(x) = StepRangeLen{Float64,Base.TwicePrecision{Float64},Base.TwicePrecision{Float64}}
typeof(x_2) = StepRange{Int64,Int64}
supertype(typeof(x)) = AbstractRange{Float64}


AbstractRange{Float64}

In [65]:
show_supertypes(typeof(x))

StepRangeLen{Float64,Base.TwicePrecision{Float64},Base.TwicePrecision{Float64}} <: AbstractRange{Float64} <: AbstractArray{Float64,1} <: Any

In [66]:
typeof(x) |> show_supertypes

StepRangeLen{Float64,Base.TwicePrecision{Float64},Base.TwicePrecision{Float64}} <: AbstractRange{Float64} <: AbstractArray{Float64,1} <: Any

In [67]:
show_supertypes(typeof(x_2))

StepRange{Int64,Int64} <: OrdinalRange{Int64,Int64} <: AbstractRange{Int64} <: AbstractArray{Int64,1} <: Any

In [68]:
@show typeof(x) <: AbstractRange
@show typeof(x_2) <: AbstractRange;

typeof(x) <: AbstractRange = true
typeof(x_2) <: AbstractRange = true


In [69]:
@show minimum(x)
@show maximum(x)
@show length(x)
@show step(x);

minimum(x) = 0.0
maximum(x) = 1.0
length(x) = 20
step(x) = 0.05263157894736842


In [70]:
f(x) = x^2
f_x = f.(x)  # calculating at the range values

@show typeof(f_x)
@show supertype(typeof(f_x))
@show supertype(supertype(typeof(f_x)))  # walk up tree again!
@show length(f_x);   # and many more

typeof(f_x) = Array{Float64,1}
supertype(typeof(f_x)) = DenseArray{Float64,1}
supertype(supertype(typeof(f_x))) = AbstractArray{Float64,1}
length(f_x) = 20


In [71]:
show_supertypes(typeof(f_x))

Array{Float64,1} <: DenseArray{Float64,1} <: AbstractArray{Float64,1} <: Any

In [72]:
? diff

search: diff symdiff setdiff symdiff! setdiff! Cptrdiff_t



```
diff(A::AbstractVector)
diff(A::AbstractArray; dims::Integer)
```

Finite difference operator on a vector or a multidimensional array `A`. In the latter case the dimension to operate on needs to be specified with the `dims` keyword argument.

!!! compat "Julia 1.1"
    `diff` for arrays with dimension higher than 2 requires at least Julia 1.1.


# Examples

```jldoctest
julia> a = [2 4; 6 16]
2×2 Array{Int64,2}:
 2   4
 6  16

julia> diff(a, dims=2)
2×1 Array{Int64,2}:
  2
 10

julia> diff(vec(a))
3-element Array{Int64,1}:
  4
 -2
 12
```


In [73]:
slopes(f_x::AbstractVector, x::AbstractRange) = diff(f_x) / step(x)

slopes (generic function with 1 method)

In [74]:
using Plots, ForwardDiff

# operator to get the derivative of this function using AD
D(f) = x -> ForwardDiff.derivative(f, x)

# compare slopes with AD for sin(x)
q(x) = sin(x)
x = 0.0:0.1:4.0
q_x = q.(x)
q_slopes_x = slopes(q_x, x)

D_q_x = D(q).(x)  # broadcasts AD across vector

plot(x[1:end-1], D_q_x[1:end-1], label = "q' with AD")
plot!(x[1:end-1], q_slopes_x, label = "q slopes")

┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1260


MethodError: MethodError: no method matching get_color_palette(::Symbol, ::RGBA{Float64}, ::Int64)
Closest candidates are:
  get_color_palette(::Any, ::Any) at /Users/taisei/.julia/packages/PlotUtils/T5XYM/src/colorschemes.jl:328
  get_color_palette(!Matched::ColorGradient, ::Any) at /Users/taisei/.julia/packages/PlotUtils/T5XYM/src/colorschemes.jl:329

In [75]:
slopes(f::Function, x::AbstractRange) = diff(f.(x)) / step(x)  # broadcast function

@show typeof(q) <: Function
@show typeof(x) <: AbstractRange
q_slopes_x = slopes(q, x)  # use slopes(f::Function, x)
@show q_slopes_x[1];

typeof(q) <: Function = true
typeof(x) <: AbstractRange = true
q_slopes_x[1] = 0.9983341664682815


In [76]:
# broadcasts over the diff
slopes(f::Function, x::AbstractArray) = diff(f.(x)) ./ diff(x)

x_array = Array(x)  # convert range to array
@show typeof(x_array) <: AbstractArray
q_slopes_x = slopes(q, x_array)
@show q_slopes_x[1];

typeof(x_array) <: AbstractArray = true
q_slopes_x[1] = 0.9983341664682815


# Exercises

## Exercise 1: StaticArrays

[StaticArrays.jl](https://github.com/JuliaArrays/StaticArrays.jl)

**StaticArrays** provides a framework for implementing statically sized arrays in Julia, using the abstract type StaticArray{Size,T,N} <: AbstractArray{T,N}. Subtypes of StaticArray will provide fast implementations of common array and linear algebra operations. Note that here "statically sized" means that the size can be determined from the type, and "static" does not necessarily imply immutable.

In [77]:
using StaticArrays 

In [78]:
using BenchmarkTools

* Example

In [79]:
v1 = SVector(1, 2, 3)

3-element SArray{Tuple{3},Int64,1,3} with indices SOneTo(3):
 1
 2
 3

In [80]:
subtypes(AbstractArray)

36-element Array{Any,1}:
 AbstractRange
 Adjoint
 Base.LogicalIndex
 Base.ReinterpretArray
 Base.ReshapedArray
 Bidiagonal
 BitArray
 CartesianIndices
 Core.Compiler.AbstractRange
 Core.Compiler.BitArray
 Core.Compiler.LinearIndices
 DataStructures.CircularBuffer
 DataStructures.DiBitVector
 ⋮
 StaticArray
 StaticArrays.TrivialView
 StatsBase.AbstractWeights
 SubArray
 SuiteSparse.CHOLMOD.FactorComponent
 SymTridiagonal
 Symmetric
 Test.GenericArray
 Transpose
 Tridiagonal
 UpperHessenberg
 ZMQ.Message

In [81]:
subtypes(StaticArray)

14-element Array{Any,1}:
 FieldVector
 GeometryTypes.AbstractPoint
 GeometryTypes.AbstractSimplex
 GeometryTypes.Face
 GeometryTypes.FixedSizeArrays.Point
 GeometryTypes.FixedSizeArrays.Vec
 GeometryTypes.Normal
 GeometryTypes.TextureCoordinate
 GeometryTypes.Vec
 MArray
 SArray
 SHermitianCompact
 SUnitRange
 SizedArray

In [82]:
supertype(AbstractArray)

Any

In [83]:
@show StaticArray <: AbstractArray

StaticArray <: AbstractArray = true


true

In [84]:
@show SArray <: StaticArray

SArray <: StaticArray = true


true

In [85]:
@show  SArray === StaticArray

SArray === StaticArray = false


false

**NOTE** : "show_supertypes() does not seem work for obejects in the class of StaticArrays.

* Benchmark VS Small Vectors/Matrices

In [86]:
N = 3
A = rand(N, N)
x = rand(N)

@btime $A * $x  # the $ in front of variable names is sometimes important
@btime inv($A)

As = @SMatrix rand(N, N)
xs = @SVector rand(N)


@btime $As * $xs  # the $ in front of variable names is sometimes important
@btime inv($As)

  67.491 ns (1 allocation: 112 bytes)
  466.429 ns (5 allocations: 1.98 KiB)
  0.024 ns (0 allocations: 0 bytes)
  6.726 ns (0 allocations: 0 bytes)


3×3 SArray{Tuple{3,3},Float64,2,9} with indices SOneTo(3)×SOneTo(3):
 -1.08923   23.2235   -8.50186
  2.27843   -6.82664   0.702451
 -0.780917  -6.9335    4.7282

In [87]:
N = 15
A = rand(N, N)
x = rand(N)

@btime $A * $x  # the $ in front of variable names is sometimes important
@btime inv($A)

As = @SMatrix rand(N, N)
xs = @SVector rand(N)


@btime $As * $xs  # the $ in front of variable names is sometimes important
@btime inv($As)

  106.733 ns (1 allocation: 208 bytes)
  3.440 μs (5 allocations: 9.88 KiB)
  0.023 ns (0 allocations: 0 bytes)
  3.576 μs (6 allocations: 11.86 KiB)


15×15 SArray{Tuple{15,15},Float64,2,225} with indices SOneTo(15)×SOneTo(15):
  0.0189847   0.428883   -0.808005   …   0.287311    -0.0471513   0.253871
 -1.51817     0.591734    0.427018       0.292501    -0.300663    0.801942
 -5.69268     1.3889     -0.243965      -1.94704      1.44634     3.47087
 -5.45545     0.201946   -0.100735      -2.89524      2.4042      4.57219
  1.57268     0.550063    0.139981       0.547999    -1.28167    -1.87514
  4.36123    -1.2114     -0.0615084  …   1.64618     -1.57242    -3.02571
  0.096034    0.122558    0.0574135      0.0384763   -0.0090053  -0.145154
  1.12936    -1.12783     0.370552       0.423491     0.547546   -0.451601
 -2.2637      0.0609243   0.175581      -1.37249      1.0732      2.33273
  4.79014    -0.593125    0.583775       2.9593      -2.25307    -3.61479
  0.225767   -0.863012   -0.552367   …  -0.00529559   0.339903    0.441463
  0.60206    -0.252941    0.309559      -0.524006     0.884211    0.448236
  0.59462    -0.744267    0.1

In [88]:
@show Float64 <: FieldVector

Float64 <: FieldVector = false


false

In [89]:
@show Float64 <: StaticVector

Float64 <: StaticVector = false


false

## Exercise 2

In [90]:
Σ = [0.4  0.3;
     0.3  0.45]
G = I
R = 0.5 * Σ

gain(Σ, G, R) = Σ * G' * inv(G * Σ * G' + R)
@btime gain($Σ, $G, $R)

  618.448 ns (10 allocations: 1.94 KiB)


2×2 Array{Float64,2}:
 0.666667     1.11022e-16
 1.11022e-16  0.666667

In [91]:
Σ = SMatrix{2,2}(0.4,0.3,0.3,0.45)
G = I
R = 0.5 * Σ

gain(Σ, G, R) = Σ * G' * inv(G * Σ * G' + R)
@btime gain($Σ, $G, $R)

  0.023 ns (0 allocations: 0 bytes)


2×2 SArray{Tuple{2,2},Float64,2,4} with indices SOneTo(2)×SOneTo(2):
 0.666667     1.11022e-16
 2.22045e-16  0.666667

In [92]:
typeof(G)

UniformScaling{Bool}

In [93]:
typeof(Σ)

SArray{Tuple{2,2},Float64,2,4}

In [94]:
typeof(R)

SArray{Tuple{2,2},Float64,2,4}

## Exercise 3

[Polynomials.jl](https://github.com/JuliaMath/Polynomials.jl)

In [95]:
using Polynomials
using Plots

p = Poly([2, -5, 2], :x)  # :x just gives a symbol for display
@show p
p′ = polyder(p)   # gives the derivative of p, another polynomial
@show p(0.1), p′(0.1)  # call like a function
@show roots(p);   # find roots such that p(x) = 0

x = -2:0.01:2
p_x = p.(x)
p′_x = p′.(x)

plot(x[1:end], p_x[1:end],label = "p") 
plot!(x[1:end-1], p′_x[1:end-1],label = "p′")

p = Poly(2 - 5*x + 2*x^2)
(p(0.1), p′(0.1)) = (1.52, -4.6)
roots(p) = [0.5, 2.0]


MethodError: MethodError: no method matching get_color_palette(::Symbol, ::RGBA{Float64}, ::Int64)
Closest candidates are:
  get_color_palette(::Any, ::Any) at /Users/taisei/.julia/packages/PlotUtils/T5XYM/src/colorschemes.jl:328
  get_color_palette(!Matched::ColorGradient, ::Any) at /Users/taisei/.julia/packages/PlotUtils/T5XYM/src/colorschemes.jl:329

## Exercise 4: Revisiting Newton's Method

In [96]:
using LinearAlgebra, BenchmarkTools,Polynomials

In [97]:
using Polynomials
function newtonsmethod(p::Poly,x_0;tolerance=1E-7,maxiter=100)
    x=x_0
    normdiff=Inf
    i = 1
    p_prime=polyder(p)
    while normdiff > tolerance && i ≤ maxiter
        x_new=x-p(x)/p_prime(x)
        normdiff=norm(x_new-x)
        x=x_new
        i += 1
    end
    return (sol=x,normdiff=normdiff,i=i)
end

newtonsmethod (generic function with 1 method)

In [98]:
using BenchmarkTools
x = Poly([2, -5, 2], :x)
@btime newtonsmethod(x,0.15)
@btime roots(x)

  255.805 ns (6 allocations: 384 bytes)
  1.181 μs (23 allocations: 2.33 KiB)


2-element Array{Float64,1}:
 0.5
 2.0

## Exercise 5: trapezoidal rule

In [99]:
using BenchmarkTools

In [122]:
function trapezoidal1(f::AbstractArray,x::AbstractArray)
    @assert length(x)==length(f)
    s=zeros(length(x)-1)
    for i in 1:length(x)-1
        s[i]=(f[i]+f[i+1])*(norm(x[i+1]-x[i]))/2
    end
    return sum(s)
end

trapezoidal1 (generic function with 1 method)

In [123]:
x=[0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
F=x.^2
trapezoidal1(F,x)

0.335

**NOTE** Can we do without vectorizing s? 

In [102]:
function tapezoidal2(f::AbstractArray,x::AbstractRange)
    @assert length(x)==length(f)
    s=zeros(length(x)-1)
    for i in 1:length(x)-1
        s[i]=(f[i]+f[i+1])*step(x)/2
    end
    return sum(s)
end

tapezoidal2 (generic function with 1 method)

In [125]:
x=0:0.1:1.0
F=x.^2
@btime tapezoidal2(F,x)

  67.411 ns (2 allocations: 176 bytes)


0.335

In [126]:
length(x)

11

In [128]:
function trapezoidal3(f::Function,a::Real,b::Real,N::Real)
    r=range(a,b,length=N)
    s=zeros(length(r)-1)
    for i in 1:length(r)-1
        s[i]=(f(r[i])+f(r[i+1]))*step(r)/2
    end
    return sum(s)
end

trapezoidal3 (generic function with 1 method)

In [166]:
sq(x::Real)=x^2
@btime trapezoidal3(sq,0.0,1.0,11)

  186.215 ns (1 allocation: 160 bytes)


0.335

**NOTE:Often functions are not defined as I intent when I use f,g. Why?**

In [106]:
using QuadGK

@btime value,accuracy=quadgk(g,0.0,1.0)

  92.996 ns (5 allocations: 144 bytes)


(1.5, 0.0)

## Exercise 6(INCOMPLETED)

In [173]:
using ForwardDiff

Df(x) = ForwardDiff.derivative(y -> , y)

Df (generic function with 1 method)

In [171]:
y=ForwardDiff.derivative(trapezoidal3(,2)

MethodError: MethodError: objects of type Float64 are not callable

In [108]:
using ForwardDiff

function 
# operator to get the derivative of this function using AD
D(f) = x -> ForwardDiff.derivative(f, x)

# example usage: create a function and get the derivative
f(x) = x^2
f_prime = D(f)

f(0.1), f_prime(0.1)

LoadError: syntax: unexpected "="

In [109]:
using BenchmarkTools,LinearAlgebra,ForwardDiff
function fixedpointmapfd(f,x_0;
        tolerance = 1e-6,maxiter = 500)
    x=x_0
    normdiff=Inf
    i = 1
    f_prime = x -> ForwardDiff.derivative(f,x)
    while normdiff > tolerance && i ≤ maxiter
        x_new=x-f(x)/f_prime(x)
        normdiff=norm(x_new-x)
        x=x_new
        i += 1
    end
    return (sol=x,normdiff=normdiff,i=i)
end

fixedpointmapfd (generic function with 1 method)